In [ ]:
import os
import pickle as pkl
import matplotlib.pyplot as plt
from matplotlib.colors import from_levels_and_colors, to_rgb
import numpy as np
import pandas as pd

In [ ]:
model_scale_list = [128, 256, 512, 1024, 1500]
model_dir_list = [
    r'D:\Users\mickey\Data\models\pyt\lfads\gw_250\lfads_ecog\cenc0_cont0_fact64_genc128_gene128_glat128_nch42_seqlen50_ulat0_orion-varstd',
    r'D:\Users\mickey\Data\models\pyt\lfads\gw_250\lfads_ecog\cenc0_cont0_fact64_genc256_gene256_glat256_nch42_seqlen50_ulat0_orion-varstd',
    r'D:\Users\mickey\Data\models\pyt\lfads\gw_250\lfads_ecog\cenc0_cont0_fact64_genc512_gene512_glat512_nch42_seqlen50_ulat0_orion-varstd',
    r'D:\Users\mickey\Data\models\pyt\lfads\gw_250\lfads_ecog\cenc0_cont0_fact64_genc1024_gene1024_glat1024_nch42_seqlen50_ulat0_orion-varstd',
    r'D:\Users\mickey\Data\models\pyt\lfads\gw_250\lfads_ecog\cenc0_cont0_fact128_genc1500_gene1500_glat1500_nch42_seqlen50_ulat0_orion-varstd',
]
psd_data_file_list = [os.path.join(s,r'figs\psd_data_dict.pkl') for s in model_dir_list]
performance_table_file_list = [os.path.join(s,r'performance_table.csv') for s in model_dir_list]
for psd_data_file in psd_data_file_list:
    assert os.path.exists(psd_data_file), f'{psd_data_file} not found.'
for perf_table_file in performance_table_file_list:
    assert os.path.exists(perf_table_file), f'{perf_table_file} not found.'

In [ ]:
def get_model_parameters_from_perf_table_path(perf_table_file):
    file_parts = perf_table_file.split(os.path.sep)
    model_dir_str = file_parts[-2]
    return model_dir_str

def read_and_concat_csv(table_file_list):
    table_list = []
    for file_idx, table_file in enumerate(table_file_list):
        model_dir_str = get_model_parameters_from_perf_table_path(table_file)
        table_row = pd.read_csv(table_file)
        table_row['model_dir_name'] = model_dir_str
        table_list.append(table_row)
    table_cat = pd.concat(table_list)
    return table_cat

perf_table_all = read_and_concat_csv(performance_table_file_list)

In [ ]:
fig, ax = plt.subplots(1,1,dpi=150)
cmap = np.linspace(to_rgb('gainsboro'),to_rgb('tab:orange'),len(psd_data_file_list))
for idx, psd_data_file in enumerate(psd_data_file_list):
    with open(psd_data_file,'rb') as f:
        psd_dict = pkl.load(f)
    ax.plot(psd_dict['f_psd'],10*np.log10(psd_dict['recon_psd_mean']),label=str(model_scale_list[idx]),color=cmap[idx,:])
ax.plot(psd_dict['f_psd'],10*np.log10(psd_dict['data_psd_mean']),color='tab:blue',label='data')
ax.plot(psd_dict['f_ar_psd'],10*np.log10(psd_dict['ar_psd_mean']),color='tab:green',label='AR')
ax.legend(loc=0,title='Unit Count')
ax.set_xlim(0,100)
ax.set_xlabel('freq. (Hz)')
ax.set_ylabel('PSD (dB)')
ax.set_title('ECoG reconstruction, varying unit count')

In [ ]:
quals_report_directory_path = r'G:\My Drive\publications\Quals paper'
fig.savefig(os.path.join(quals_report_directory_path,'psd_vary_unit_count.png'))
fig.savefig(os.path.join(quals_report_directory_path,'psd_vary_unit_count.svg'))

In [ ]:
ar_rpe_mean = 1.02
ar_rpe_lb = 1.01
ar_rpe_ub = 2.67
ar_corr_mean = 0.25
ar_corr_lb = 0.18
ar_corr_ub = 0.34
fig, ax = plt.subplots(1,1,dpi=150)
# ax.axhspan(ar_rpe_lb,ar_rpe_ub,alpha=0.2,color='tab:green')
# ax.axhline(ar_rpe_lb,color='tab:red',linestyle='--')
# ax.axhline(ar_rpe_ub,color='tab:red',linestyle='--')
# ax.axhline(ar_rpe_mean,color='tab:red',linestyle='-')
ax.errorbar(model_scale_list,perf_table_all['mse_mean'],
            [perf_table_all['mse_97.5ci']-perf_table_all['mse_mean'],
             perf_table_all['mse_mean']-perf_table_all['mse_2.5ci']],
            capsize=2.5, color='tab:red', marker='o',
            label='MSE')
ax.errorbar(0,ar_rpe_mean,[[ar_rpe_mean-ar_rpe_lb],[ar_rpe_ub-ar_rpe_mean]],
            capsize=2.5, color='tab:red', marker='o', markerfacecolor='tab:green', markeredgecolor='tab:green')
# corr
ax.errorbar(model_scale_list,perf_table_all['corr_mean'],
            [perf_table_all['corr_97.5ci']-perf_table_all['corr_mean'],
             perf_table_all['corr_mean']-perf_table_all['corr_2.5ci']],
             capsize=2.5, color='tab:purple', marker='o',
             label='Corr.')
ax.errorbar(0,ar_corr_mean,[[ar_corr_mean-ar_corr_lb],[ar_corr_ub-ar_corr_mean]],
            capsize=2.5, color='tab:purple', marker='o', markerfacecolor='tab:green', markeredgecolor='tab:green')
ax.legend(loc=0)
ax.set_xticks([0, 200, 400, 600, 800, 1000, 1200, 1400])
ax.set_xticklabels(['AR', '200', '400', '600', '800', '1000', '1200', '1400'])
ax.set_xlabel('# of Units')
ax.set_ylabel('Metric Measurement')
ax.set_title('Reconstruction Error and Corr. v. Unit Count')

In [ ]:
quals_report_directory_path = r'G:\My Drive\publications\Quals paper'
fig.savefig(os.path.join(quals_report_directory_path,'perf_vary_unit_count.png'))
fig.savefig(os.path.join(quals_report_directory_path,'perf_vary_unit_count.svg'))